# DATA Scraping and Extraction for four agencies

# Data Collection

## NIH DATASET COLLECTION

Beacuse I can only extract data of information of 15000 projects for a single time, I extracted all projects for 6 times.

### (1) Import NIH datasets 

In [1]:
## Import NIH datasets
# NIH1 = pd.read_csv("../Data/NIH/NIHFundedData1.csv")
# NIH2 = pd.read_csv("../Data/NIH/NIHFundedData2.csv")
# NIH3 = pd.read_csv("../Data/NIH/NIHFundedData3.csv")
# NIH4 = pd.read_csv("../Data/NIH/NIHFundedData4.csv")
# NIH5 = pd.read_csv("../Data/NIH/NIHFundedData5.csv")
# NIH6 = pd.read_csv("../Data/NIH/NIHFundedData6.csv")

### (2) Concatenate all datasets

In [7]:
# frames = [NIH1, NIH2, NIH3, NIH4, NIH5, NIH6]
# df = pd.concat(frames)

### (3) Save dataset

In [8]:
# f.columns = df.columns.map(lambda x: x.replace(" ", ""))
# df.to_csv("../Data/NIHFundedData.csv")

## NSF DATASET COLLECTION

I first downloaded all xml files, where each file contain information of one project. So I first obtained all xml filenames and iterated them to extract info of each project. Then the final dataset will be saved into local directory.

### (1) Extract NSF info from all xml-format files

In [10]:
def getNSFDataset(amt = None, path = "/Users/kyle/Documents/IC/ALL_Courses/Final_Projects/Data/NSF/", verbose = False):
    """ Parse XML file to get all NSF funded data """
    ## Getting all xml filenames we have
    subdirnames = []
    filenames = []
    for (dir, subdir, files) in os.walk(path):
        for names in subdir:
            subdirnames.append(names)
        for name in files:
            if name.split(".")[1] == "xml":
                filenames.append(os.path.join(dir, name))
                
    amt = len(filenames) if amt == None else amt
    
    ## Processing xml files
    for fileNum in tqdm(range(amt)):
        if verbose:
            print("Processing number {}, file {}".format(fileNum, filenames[fileNum].split("/")[10]))
        try:
            tree = et.parse(filenames[fileNum])
            root = tree.getroot()
        except Exception as msg:
            if re.match('no element found', str(msg)):
                print("Loading file {} failed".format(filenames[fileNum].split("/")[10]))
                pass
            else:
                raise ValueError("Parse failed in file number {}, the file path is {}".format(fileNum, filenames[fileNum]))

        ## to iterate the xml formatted 'tree'
        dic = {}
        for elem in root:
            for subelem in elem:
                if subelem.text == '\n':
                    for subelem2 in subelem:
                        if subelem2.text == '\n':
                            for subelem3 in subelem2:
                                name = "{}_{}".format(subelem3.tag, subelem2.tag)
                                dic[name] = [subelem3.text]
                        else:
                            name = "{}_{}".format(subelem2.tag, subelem.tag)
                            dic[name] = [subelem2.text]
                else:
                    dic[subelem.tag] = [subelem.text]
        dic["FiscalYear"] = filenames[fileNum].split("/")[9]
        if 'DRECONTENT_POR' in dic:
            del dic['DRECONTENT_POR'] 
        
        ## Removing punctuations for AwardTitle, AbstractNarration and POR_COPY_TXT_POR columns
        dic['AwardTitle'] = [remove_punctuation(dic['AwardTitle'][0])] if dic['AwardTitle'][0] != None else dic['AwardTitle']
        dic['AbstractNarration'] = [remove_punctuation(dic['AbstractNarration'][0])] if dic['AbstractNarration'][0] != None else dic['AbstractNarration']
        if 'POR_COPY_TXT_POR' in dic:
            dic['POR_COPY_TXT_POR'] = [remove_punctuation(dic['POR_COPY_TXT_POR'][0])] 

        if fileNum == 0:
            df = pd.DataFrame(dic)
        else:
            df = df.append(dic, ignore_index=True)
    return df

In [11]:
# %%time
# df = getNSFDataset(path = "/Users/kyle/Documents/IC/ALL_Courses/Final_Projects/Data/NSF/", verbose=False)

### (2) A Little Processing

In [12]:
def remove_list(x):
    """ Romving list of each elemnt within dataframe """
    for idx in range(len(x)):
        if type(x[idx]) == list:
            x[idx] = x[idx][0]
        else:
            x[idx] = x[idx]
    return "Finished!"

In [13]:
# %%time
# df.apply(remove_list, axis = 1)
# df

In [14]:
# df = df[["AwardTitle", "FiscalYear", "AGENCY", "AwardEffectiveDate", "AwardExpirationDate", 
#     "AwardTotalIntnAmount", "AwardAmount", "AbstractNarration", "MinAmdLetterDate", "MaxAmdLetterDate",
#     "TRAN_TYPE", "CFDA_NUM", "NSF_PAR_USE_FLAG", "FUND_AGCY_CODE", "AWDG_AGCY_CODE", "AwardID", "FUND_OBLG"]]

### (3) Save dataset

In [15]:
# df.to_csv("../Data/NSF_Funded_Data.csv")

## UKRI ABSTRACT COLLECTION

I first downloaded information of all projects without abstract data. Then I used GtR api to extract all abstarct data for each project.

### (1) Create a class for accessing MySQL database

In [24]:
class MySQLPipline(object):
    """ Define Class for MySQL Connection"""
    def __init__(self, database = 'funding'):
        """ Initialize object """
        self.conn = pymysql.connect(
            host = 'localhost', port = 3306, user = 'root', passwd = 'Kyle9975', db = self.database, charset = 'utf8')
        self.conn.autocommit(True)
        self.cursor = self.conn.cursor()

    def process_Query(self, sql, colnames):
        """ Processing the SQL query"""
        self.cursor.execute(sql)
        data = self.cursor.fetchall()
        data = pd.DataFrame(data)
        data.columns = colnames
        return data
    
    def INSERT_Query_UKRI(self, item):
        sql = 'INSERT INTO UKRI_Funded_ALL_Raw (TITLE, ProRef, OrgName, sDate, eDate, institution, department, projType, PIFirstName, PISurname, Amount, url, Abstract, resSub, resTopic) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'
        iteminfo = [item['title'][0], item['proRef'][0], item['orgName'][0], item['sDate'][0], item['eDate'][0], item['institution'][0], item['department'][0], item['projType'][0], item["PIFirstName"][0], item["PISurname"][0], item["Amount"][0], item["url"][0], item["abstract"], str(item["resSub"]), str(item['resTopic'])]
        self.cursor.execute(sql, iteminfo)
        return item
    
    def INSERT_Query_ERC(self, itemInfo):
        sql = 'INSERT INTO ERC_NEW (project_number, title, abstract, grant_type, topic, amount, project_ID, sDate, eDate, institution) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'
        iteminfo = itemInfo
        self.cursor.execute(sql, iteminfo)
        return itemInfo

    def close_Conn(self):
        """ Closing Connection """
        self.cursor.close()
        self.conn.close()

### (2) Scraping project abstracts using GtR API

In [3]:
def getAbstract(row):
    """ Get abstract for each funded project and directly save them into MySQL database """
    dic = row.to_dict('list')
    ref = dic["proRef"][0]
    
    while True:
        try:
            response = requests.get("https://gtr.ukri.org/projects?ref={}".format(ref),
            headers = {
                'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
            })
            break
        except:
            pass
    
    absPattern = re.compile("<gtr:abstractText>(.*?)</gtr:abstractText>", re.S)
    resAbs = re.findall(absPattern, response.text)
#     print(resAbs)
    dic["abstract"] = resAbs[0] if len(resAbs) != 0 else None
    
    subPattern = re.compile("<gtr:researchSubject>.*?<gtr:text>(.*?)</gtr:text>.*?</gtr:researchSubject>", re.S)
    resSub = re.findall(subPattern, response.text)
    dic["resSub"] = resSub if len(resSub) != 0 else None
    
    topicPattern = re.compile("<gtr:researchTopic>.*?<gtr:text>(.*?)</gtr:text>.*?</gtr:researchTopic>", re.S)
    resTopic = re.findall(topicPattern, response.text)
    
    if len(resTopic) == 0:
        resTopic = None
    else:
        if resTopic[0] == "Unclassified":
            resTopic = resTopic[0]
            
    
    dic["resTopic"] = resTopic
    
    UKRI = MySQLPipline()
    UKRI.INSERT_Query_UKRI(dic)
    UKRI.close_Conn()
    
    return dic

In [26]:
# for i in tqdm(range(UKRIdata.shape[0])):
#     getAbstract(UKRIdata[UKRIdata.index == i])

## ERC DATA COLLECTION

I first downloaded part of infomation of all projects, then I scraped other info that is not exist in my downloaded file from websites and directly save them into MySQL database.

In [27]:
# # Get all project IDs
# count = 0
# projectID = []
# all_projects = []
# for i in [1,2,3,4,5]:
#     response = requests.get("https://erc.easme-web.eu/json/list-{}.json".format(i),
#     headers = {
#         'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
#     })
#     list_ = response.json()
#     for elem in list_['items']:
#         for project in elem["projects"]:
#             all_projects.append(project)
#             projectID.append(project['number'])

In [28]:
def scrape_ERC_data(projectID, allproject):
    """ Scraping project info. like start and end date, project ID and funding amount etc."""
    title = []
    abstract = []
    grant_type = []
    topic = []
    EU_contribution = []
    Project_IDs = []
    sDate = []
    eDate = []
    institutions = []

    for idx, id_ in enumerate(tqdm(projectID)):
#         print("request id {}...".format(id_))
        sleep(0.5)
    
        while True:
            try:
                response = requests.get("https://erc.easme-web.eu/json/projects/project-{}.json".format(id_),
                        headers = {
                            'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
                        })
                break
            except:
                print("connection time out! reconnecting...")
                pass

        regex = """.*?<div><label>Project Title:</label> <span>(.*?)</span></div>""" + \
                """.*?<p>(.*?)</p>""" + \
                """.*?<div><label>Grant Type:</label> <span>(.*?)</span></div>""" + \
                """.*?<div><label>Topic:</label> <span>(.*?)</span></div>""" + \
                """.*?<div><label>EU Contribution:</label> <span>(.*?)€</span></div>""" + \
                """.*?<div><label>Call ID:</label> <span>(.*?)</span></div>""" + \
                """.*?<div><label>Start date:</label> <span>(.*?)</span></div>""" + \
                """.*?<div><label>End date:</label> <span>(.*?)</span></div>"""

        regex_host_institutions = '''.*?title="(.*?)">'''
        
        projectInfo_dic = allproject[idx]

        pattern = re.compile(regex, re.S)
        pattern_host_institutions = re.compile(regex_host_institutions)
        res = re.findall(pattern, response.text)
        host_institutions = [ elem.strip() for elem in re.findall(pattern_host_institutions, response.text) ]
        
        itemInfo = [
            projectInfo_dic['number'],
            res[0][0],
            res[0][1],
            res[0][2],
            projectInfo_dic['topic'],
            projectInfo_dic['budget'],
            res[0][5],
            res[0][6],
            res[0][7],
            str(host_institutions)
        ]
        
#         print(itemInfo[0])
        
        UKRI = MySQLPipline()
        UKRI.INSERT_Query_ERC(itemInfo)
        UKRI.close_Conn()
        
    return 0

In [29]:
# scrape_ERC_data(projectID, all_projects)